# Load the modules needed 

In [1]:
# Basic Function Modules
import sys
import os
import glob
from imp import reload
import csv
import threading
from pathlib import Path
import warnings
#Data Analysis Modules
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Image Detection Modules
import skimage.measure as measure
from scipy.ndimage import label, distance_transform_edt
from skimage import io, img_as_float, filters, morphology
from skimage.feature import peak_local_max
from skimage.measure import regionprops, regionprops_table
from skimage.morphology import watershed
# Set the working directory to the AutomatedCE folder so we have access to L1-L4 modules
resp = os.getcwd()
if 'testing' in resp[-7:]:
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
    print(f"new directory is: {os.getcwd()}")
#AutomatedCE Modules
import testing.inputs  
from testing.inputs import *
from L4 import DisplayWindows
from L4 import FileIO
from L4 import Focus
from L4 import AutomatedControl
from L4 import Trajectory
from L4.image_util import ImageSaver
from L3 import SystemsBuilder
import testing.te300_auto_utils as auto_utils
from L4 import image_util
from L4 import FileIO
from L4 import CellDetection
%config Completer.use_jedi = False
 
# Reload (when changing aspects in the following modules)
reload(Focus)
reload(DisplayWindows)
reload(testing.inputs)
import warnings
warnings.simplefilter("ignore", UserWarning)
# Make the plots pop up in a separate window
%matplotlib qt
counter=0

new directory is: C:\Users\NikonTE300CE\Desktop\Barracuda_Updated\AutomatedCE
C:\Users\NikonTE300CE\Desktop\Barracuda_Updated\AutomatedCE\L1\MicroControlServer.py


## Step 1. Load the CE System with a Config File

Select one of the predesigned configurations (TE300, Eclipse Ti, Simulation). See an example config under "/config/test-system.cfg". 

Change the variable CONFIG_PATH to the absolute path where the config file resides. 

Be sure to make the string a raw string by prepending *'r'*. **Example:** r"C:\path\to\config\"*

If you get an error, you must restart the kernel before re-running. 

In [2]:
#CONFIG_PATH = r"C:\Users\Luke\Desktop\Barracuda\AutomatedCE\var\Eclipse.cfg"
CONFIG_PATH = r"C:\Users\NikonTE300CE\Desktop\Barracuda_Updated\AutomatedCE\config\TE300.cfg"
#CONFIG_PATH = r"D:\Scripts\CEInstrument\AutomatedCE\config\Test-System.cfg"
reload(SystemsBuilder)

# Create the system object
ce_system = SystemsBuilder.CESystem()
# Read the config file
ce_system.load_config(CONFIG_PATH)
# Open the controllers and run the startup functions
ce_system.open_controllers()
ce_system.startup_utilities()

# Adjust the poorly wired RGB lighting lol
ce_system.inlet_rgb.turn_on_channel('B')
ce_system.inlet_rgb.turn_off_channel('R')

# Adjust the excitation wheel start position
ce_system.filter_wheel.set_channel(1)
ce_system.camera.set_exposure(100)

pulses=0

daq
newwave
{'outlet_pressure': <L2.PressureControl.ArduinoPressure object at 0x000001E93A6B14C0>, 'xy_stage': <L2.XYControl.PriorXY object at 0x000001E93A6B1520>, 'objective': <L2.ZControl.PriorZ object at 0x000001E93A6B1580>, 'outlet_z': <L2.ZControl.ArduinoZ object at 0x000001E93A6B15B0>, 'inlet_rgb': <L2.LightControl.RGBArduino object at 0x000001E93A6B1550>, 'inlet_z': <L2.ZControl.KinesisZ object at 0x000001E93A6B1640>, 'high_voltage': <L2.HighVoltageControl.SpellmanPowerSupply object at 0x000001E93A6B1610>, 'camera': <L2.CameraControl.PycromanagerControl object at 0x000001E93A6B16A0>, 'detector': <L2.DetectorControl.PhotomultiplierDetector object at 0x000001E93A6B1AC0>, 'lysis_laser': <L2.LaserControl.NewWaveBNC object at 0x000001E93A6B1D30>, 'excitation_wheel': <L2.FilterWheelControl.LumencorFilter object at 0x000001E93A6B1D00>, 'filter_wheel': <L2.FilterWheelControl.PriorFilter object at 0x000001E93A6B1CA0>}
OPENING:  ard1 <L1.Controllers.ArduinoController object at 0x000001E93

In [3]:
# Turn off LED
ce_system.inlet_rgb.turn_off_channel('B')


'R:Off, G:Off, B:Off'

In [4]:
# Turn on LED
ce_system.inlet_rgb.turn_on_channel('B')

'R:Off, G:Off, B:On'

In [5]:
# Random Globals Need to Run for Powerpoint!
pulses=0
cell_notes = ''

# Record lysis images? 
record_lysis = True
#lysis_dir = r"C:\Users\NikonTE300CE\Desktop\Luke\Lysis\images"
#lysis_dir = r"C:\Users\NikonTE300CE\Desktop\Mani\Lysis\images"
lysis_dir = r"C:\Users\NikonTE300CE\Desktop\Luke\Lysis\images"

# Step 2. Prepare the Capillary for Electrophoresis

We need to prep the capillary for CE. This involves rinsing the capillary surface and removing contaminants as well as re-establsihing a deprotonated silanol layer. We also need to align the capillary with detection objective. 

### Step 2.1 Rinse the capillary (Repeatable)

Rinse the capillary according to your method. A good place to start is 10 min with HCl, 30 min with NaOH, 2 min with water. If this is your first rinse of the day,rinse with an empty vial first to make sure there are no leaks on the outlet side of the apparatus. 

In [4]:
# Start Rinse
ce_system.outlet_pressure.rinse_pressure()

In [477]:
# Stop Rinse 
ce_system.outlet_pressure.release()

### Step 2.2 Align the capillary with fluorescein. 

Place a vial with 125 nM fluorescein in the outlet. Once the pressure is rinsing adjust the capillary position using the two set screws as well as the adjustable objective collar until the signal is maximized for the detector being used. (PMT gain voltage should be set to 250 mV, or the lowest setting with appreciable gain). 

In [47]:
# Run the CE View
reload(DisplayWindows)

# Create a CE Display
ce_disp = DisplayWindows.PLTCEDisplay(ce_system)
ce_disp.show()
ce_system.detector.start()
ce_disp.start_live_view()

In [17]:
resp = get_yes_no("Is fluorescein in the outlet and ready to be rinsed with?")

if resp:
    ce_system.outlet_pressure.rinse_pressure()
    print("Adjust the capillary inlet while maximizing the signal")
if not resp:
    print("Please insert the fluorscein in the outlet, then re-run this cell.")

Is fluorescein in the outlet and ready to be rinsed with? y
Adjust the capillary inlet while maximizing the signal


When finished. Record the maximum signal, along with the detector used, its gain setting (or bias when using SiPM), and the fluorescein concentration in the calibration log (located on the desktop). 

Run the next cell to stop the pressure rinse. 

In [25]:
ce_system.outlet_pressure.release()

XY Errr ['']
After special True
Time to wait is  5
XY Errr ['']
After special True
Time to wait is  100
XY Errr ['']
After special True
Time to wait is  5
XY Errr ['']
After special True
Time to wait is  100
XY Errr ['']
After special True
Time to wait is  5
XY Errr ['']
After special True
Time to wait is  100
XY Errr ['']
After special True
Time to wait is  5
XY Errr ['']
After special True
Time to wait is  100


### Step 2.2 Post Fluorescein Rinse

Take a vial and add close to 4 mL of H20. Insert it into the outlet and then remove it. This removes any residual Fluorescein from the capillary, electrode, or other surfaces.  

Repeat with a second vial, with liquid slightly higher or the same height. 

Finally switch with separation buffer and rinse for 10 min. 

In [20]:
#Optional Step
resp = get_yes_no("Have you done the pre-fluorescein rinses?")
if not resp:
    print("Please follow instructions in step 2.2")
else:
    resp = get_yes_no("Is the separation buffer placed in the outlet?")
    if resp:
        ce_system.outlet_pressure.rinse_pressure()
        print("Run the next cell when finished")
    else:
        print("Please place the separation buffer in the outlet before rinsing.")

Have you done the pre-fluorescein rinses? y
Is the separation buffer placed in the outlet? y
Run the next cell when finished


In [25]:
ce_system.outlet_pressure.release()

# Step 3. Calibrate the Capillary Inlet

The method for calibration depends on the z-stage used to raise the capillary inlet. Select the correct route for the hardware employed. 

## Thorlabs Kinesis Z Stage

### Step 3.1 Lower the Capillary (thorlabs)
Move the capillary to a safe location where it will not be crushed if the stage lowers too far. Using the controls on the Z-stage itself, lower the capillary till as far as it will go. 

In [5]:
resp = get_yes_no("Have you lowered the Kinesis Z stage to its lowest position? (y/n):  ")
if not resp:
    print("Please make sure the Kinesis stage is lowered to its lowest position before continuing")
else:
    print("Proceed to the next cell.")
    

Have you lowered the Kinesis Z stage to its lowest position? (y/n):   y
Proceed to the next cell.


### Step 3.2 Home the Capillary (thorlabs)

Thorlabs needs to register the home position. In the next cell we will call the go_home command to home the capillary. 

In [6]:
ce_system.inlet_z.homing()

### Step 3.3 Adjust Capillary Position (all stages)

Adjust the capillary position by sliding the detection apparatus up or down the optical support post. The capillary should be level with the bottom of the template for single-cell assays, or the top-of the template for well-based assays. 

*Aligning to the top of the template is a minor safety feature to prevent the user from sending the capillary to Zero position unexpetedly and crashing the capillary against the bottom of the template. However, capillary needs to be able to reach at least the glass slide of the cell reservoir for single-cell assays*

In [7]:
resp = get_yes_no("Have you adjusted the capillary height manually so that it is flush (level) with the top or bottom of the template? (y/n):  ")
if not resp:
    print("Please make sure the capillary is moved to the correct height with respect to the template before continuing")
else:
    print("Proceed to the next cell")

Have you adjusted the capillary height manually so that it is flush (level) with the top or bottom of the template? (y/n):   y
Proceed to the next cell


### Step 3.4 Raise the capillary to a safe height

The next cell will raise the capillary height to a location where it ***should*** be safe for any XY movements. Verify the capillary will not crash at this new height. 

In [8]:
target_z = get_float("Please enter a safe height to move the stage to: ")
# Move the capillary up and wait for it to stop
ce_system.inlet_z.set_z(target_z)
ce_system.inlet_z.wait_for_target(target_z)

resp = get_yes_no("Is the capillary at a safe location?: ")
if resp:
    print("Continue to next step")
else:
    print("Please raise the capillary to an appropriate height")

Please enter a safe height to move the stage to:  26
Is the capillary at a safe location?:  y
Continue to next step


# Step 4. Home the XY Stage

This next step provides the calibration for the XY axis for the XY stage and capillary inlet. 


### Step 4.1 (TE300, Prior)

The TE300 does not have the encoders wired to tell it where absolute 0,0 is. Thus, we need to tell the microcontroller where 0,0 is for each template. The home position before any offset, will be when the objective is in the top-right corner (if looking down at it). 

1. Move the objective to the upper right corner of the template. 
2. Align the capillary over the well or the objective


In [24]:
resp = get_yes_no("Is the capillary aligned with the objective and home well?")
if resp:
    ce_system.xy_stage.set_home()
    print("Home set at this position.")
    print("Continue to next step")
else:
    print("Please raise align the capillary with the objective")   

Is the capillary aligned with the objective and home well? y
Home set at this position.
Continue to next step


# Step 5 Verify your calibration

Steps 3-4 calibrate the capillaries position with the microscope's position. Now we need to verify that our alignment is correct. To do this we will move the capillary to a well and make sure that it is centered over the well. Then we will lower the capillary to a height specified by the method to make sure the capillary can reach the liquid. 

### Step 5.1. Load the Template File

The template file tells the program where to move the XY stage for each well as well as how high to raise the capillary inlet during a move. Change the variable TEMPLATE_PATH to the absolute path where the template.txt file resides. 

In [6]:
TEMPLATE_PATH = r"C:\Users\NikonTE300CE\Desktop\Barracuda_Updated\AutomatedCE\var\TE300CellTemplate.txt"

reload(AutomatedControl)
reload(Trajectory)

auto_program = AutomatedControl.AutoRun(ce_system)
auto_program.set_template(TEMPLATE_PATH)

### Step 5.2a Raise the capillary to a safe height 

In [285]:
z = get_float("Set the capillary to a safe travel height: ")
ce_system.inlet_z.set_z(z)
print("Proceed to next cell")

Set the capillary to a safe travel height:  25
Proceed to next cell


### Step 5.2b Center the Capillary over a well

In [25]:
name = get_value("Please enter the well name to move to (press enter to see a list): ", auto_program.template.wells.keys())
well = auto_program.template.wells[name]
ce_system.xy_stage.set_xy(well.xy)
resp = input("Please move the capillary using the fine adjustment knobs, or the z-stage base so that capillary is centered over this well.")


Please enter the well name to move to (press enter to see a list): w_a0
Please move the capillary using the fine adjustment knobs, or the z-stage base so that capillary is centered over this well.y


### Step 5.2c Adjusting the capillary center software offset.
**(you can run this after aligning the cap to the objective for better centering over vials)**
Occasionally, when the capillary is aligned over the objective, it will no longer be aligned over the vials. The next commands allow you account for this slight shift. Unlike the calibration steps above, this step allows you to move the XY-stage to center the well and keep the capillary stationary. This is most important for when the electrode or capillary look as if they may hit the vial walls during the run. 

#### For single cell runs, run this offset after the capillary has been aligned to the objecive using the micromanipulators. 


In [60]:
# If you had to create a new 'auto_program' object, run this command to run the xy_offset from earlier.
try:
    auto_program.offset = xy_offset
except:
    print("XY Offset not set")

In [12]:
# Run this command to create a new XY_offset 
name = get_value("Please enter the well name to move to (press enter to see a list): ", auto_program.template.wells.keys())
well = auto_program.template.wells[name]
auto_program.move_to_well(name)
resp = get_yes_no("Move the capillary using the xy stage joystick, or the z-stage base so that capillary is centered over this well. \n Is the capillary centered?")
if resp: 
    xy_offset = auto_program.offset_template(well.xy)

Please enter the well name to move to (press enter to see a list): w_a2
Move the capillary using the xy stage joystick, or the z-stage base so that capillary is centered over this well. 
 Is the capillary centered? y
Offset is: [2.7420000000000044, -1.4601428571428556]


### Step 5.3 Lower the capillary to the height described in the method file


In [47]:
height_z = get_float("What height should the capillary be lowered to? (in mm, use inlet height from methods file)")
prev_height = ce_system.inlet_z.read_z()
ce_system.inlet_z.set_z(height_z)
resp = get_yes_no("Is the capillary touching the liquid in this well? ")
if not resp:
    print("Adjust the height of the capillary manually (and carefully!) so that the capillary is touching the liquid")
else:
    print("Calibration Finished")
ce_system.inlet_z.set_z(prev_height)

What height should the capillary be lowered to? (in mm, use inlet height from methods file) 1.5
Is the capillary touching the liquid in this well?  y
Calibration Finished


### Optional> 
Read out the height you are currently at...

In [63]:
print("You are at {} mm".format(ce_system.inlet_z.read_z()))

You are at 5.56536 mm


In [ ]:
resp = get_yes_no("Would you like to lower the capillary?")
if resp:
    resp = get_float("What would you like to lower it to?")
    ce_system.inlet_z.set_z(resp)
else:
    pass

# Step 6. Load a method

### NOTE: Step 5.1 must be run before this 

Load a method file. Verify your method file in excel and make sure the run is set up as intended. You can add multiple files to the same auto_program

If you need to reset the methods (remove and start over) run the next cell

In [13]:
# Clears the methods to be added to the run protocol
auto_program.methods=[]

In [14]:
# Put the filepath to your method here:
file_to_add = r"C:\Users\NikonTE300CE\Desktop\Barracuda_Updated\AutomatedCE\var\Luke/SingleCell.txt"
#file_to_add = r"C:\Users\NikonTE300CE\Desktop\Barracuda_Updated\AutomatedCE\var\Luke/SingleCell.txt"
# Reads the method, adds it to our list of methods to run
auto_program.add_method(file_to_add)

WHASSUZ ['separate\tcells\t21 cm\t10 mm\t0\t0\t10 kV\t3 s\twait\t0', '20210501_LysisTesting\tw_a1\t21 cm\t10 mm\t0\t0\t20 kV\t5 s\t\t1', '1M NaOH\tw_a2\t21 cm\t10 mm\t0\t1\t0\t15 s\t\t0', 'pressure rinse\tw_a3\t21 cm\t10 mm\t1\t0\t0\t5 s\t\t0']
ADDING
ADDING
ADDING
ADDING


In [15]:
# Set the number of repetitions to run each method
auto_program.repetitions=45

# Setup Powerpoint Output

In [16]:
reload(auto_utils)
reload(image_util)
reload(FileIO)
SlideSingleCell = FileIO.SlideSingleCell

In [17]:
# Enter power point file name here
file_name = '20210508_Luke_BraeLysisTest'

# Ask 
resp = get_yes_no("Do you want to save the old presentation (if applicable) and start a new record?\n"
                  "Previous Presentations will not be overwritten. \n" 
                  "Slides will be saved to the Data Folder")
if resp:
    cell_notes = ""
    file_name = FileIO.get_data_filename(file_name+'.pptx', auto_program.data_dir, extension=".pptx")
    slide_show = SlideSingleCell(file_name)
    last_lysis = None

Do you want to save the old presentation (if applicable) and start a new record?
Previous Presentations will not be overwritten. 
Slides will be saved to the Data Folder y


# Automated Run Start and Stop Commands

In [18]:
# Start the run
auto_program.start_run()

In [68]:
# Stop the Run
auto_program.stop_run()
auto_program.continue_event.set()

AttributeError: 'AutoRun' object has no attribute 'co'

ERROR:root:Error while moving during System Move. Aborting
ERROR:root:Exiting the run...


In [50]:
# Run to continue past a wait step
auto_program.continue_event.set()

# Displays

In [85]:
# CE Display
ce_gram = DisplayWindows.PLTCEDisplay(ce_system)
#ce_disp.stop()
#ce_disp.show()
ce_gram.start_live_view()

# Uncomment the following to reset the detector (start gathering data)
#ce_system.detector.start()

After special True
Time to wait is  300
XY Errr ['']
After special True
Time to wait is  3
XY Errr ['']
After special True
Time to wait is  300
XY Errr ['']
After special True
Time to wait is  3
After special True
Time to wait is  300
After special True
Time to wait is  3
After special True
Time to wait is  300
XY Errr ['']
After special True
Time to wait is  3
XY Errr ['']
After special True
Time to wait is  300
After special True
Time to wait is  3
After special True
Time to wait is  300
After special True
Time to wait is  3
After special True
Time to wait is  300
XY Errr ['']
After special True
Time to wait is  3
XY Errr ['']
After special True
Time to wait is  300
XY Errr ['']
After special True
Time to wait is  3
After special True
Time to wait is  300


In [54]:
# Microscope Display
cam = DisplayWindows.PLTMicroscopeDisplay(ce_system)
cam.scalar=1
ce_system.camera.stop()
cam.show()
cam.live_image()

HEYYOO


# SET DISTANCE BETWEEN OBJECTIVE AND CAPILLARY

In [27]:
# If this is not 0, make sure the objective won't crash!
ce_system.objective.read_z()

0.0

In [26]:
# Go to Zero at the start of the day
ce_system.objective.set_z(0)

Now bring cells into focus, by using the coarse objective adjustment! Do not change the coarse adjustment from here on out.

In [83]:
# Move the objective to the cell focus, and run this cell to record it for next time
obj_height = ce_system.objective.read_z()
ce_system.objective.set_rel_z(-0.05)

In [82]:
# Lower Capillary Inlet into Cell Chamber units in mm
ce_system.inlet_z.set_rel_z(-0.005)

In [84]:
# and record this height for next time 
cap_height = ce_system.inlet_z.read_z()
cap_difference = obj_height - cap_height
with open('save_me','w') as fout:
    fout.write(f'cap_difference,{cap_difference}')

In [8]:
# if you needed to restart you can load the earlier difference
with open('save_me','r') as fin:
    cap_difference= float(fin.readline().split(',')[1])
    

In [82]:
ce_system.inlet_z.read_z()

14.168074

In [85]:
ce_system.inlet_z.set_rel_z(5)

In [55]:
ce_system.objective.set_rel_z(-0.025)

# Take Fluorescent Images

This is for the NikonTE300 Microscope. 

*light_channel* corresponds to the lumencor light. Please enter the lowercase name of light to show:
'red', 'blue', 'cyan', etc...

*fluor_channel* corresponds to the filter wheel position for the emission filter. 

GFP Fluoresence settings:
light_channel = 'cyan'
fluor_channel = 4


In [76]:
fluor_channel=4
light_channel='cyan'
bins=2
ce_system.excitation_wheel.set_intensity(['cyan'],1)
auto_utils.pre_fluoresence(ce_system)
img = ce_system.camera.snap()
auto_utils.post_fluoresence(ce_system)



HEYYOO


In [77]:
# Show the image you just took 
fig, ax = plt.subplots()
ax.imshow(img)

In [71]:
# Show the histogram of the image you just took 
fig,ax = plt.subplots()
ax.hist(img.flatten())

(array([ 3718., 12182.,  3998.,  1532.,   633.,   326.,   129.,    55.,
           33.,    14.]),
 array([192. , 206.2, 220.4, 234.6, 248.8, 263. , 277.2, 291.4, 305.6,
        319.8, 334. ]),
 <BarContainer object of 10 artists>)

# SINGLE CELL 

In [287]:
# DO NOT RUN THIS BOX FOR THE FIRST CELL OR YOU WILL WRECK THE CAPILLARY.
#FOR FIRST CELL, RUN "FIND YOUR CELLS OF INTEREST, LOWER THE CAPILLARY"
#Run this box, when you need to raise the objective up, following a single cell. 
resp = get_yes_no("You want to raise the objective? DO NOT RUN IF FIRST CELL.")
if resp:
    ce_system.objective.set_rel_z(5)

You want to raise the objective? DO NOT RUN IF FIRST CELL. y


Exception in thread Thread-135364:
Traceback (most recent call last):
  File "C:\Users\NikonTE300CE\miniconda3\envs\CEpy37\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\NikonTE300CE\miniconda3\envs\CEpy37\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\NikonTE300CE\Desktop\Barracuda_Updated\AutomatedCE\L2\CameraControl.py", line 240, in _sequence_update
    img = self.controller.send_command(self.controller.core.get_last_image)
  File "C:\Users\NikonTE300CE\Desktop\Barracuda_Updated\AutomatedCE\L1\Controllers.py", line 232, in send_command
    raise e
  File "C:\Users\NikonTE300CE\Desktop\Barracuda_Updated\AutomatedCE\L1\Controllers.py", line 223, in send_command
    ans = command(*args)
  File "C:\Users\NikonTE300CE\miniconda3\envs\CEpy37\lib\site-packages\pycromanager\core.py", line 399, in <lambda>
    ): instance._translate_call(signatures_list, args)
  File "C:\Users\NikonTE300CE\miniconda3\envs\

In [93]:
# Use these two commands to record, or go to previous locations on the glass slide. 
#xy = ce_system.xy_stage.read_xy()
#ce_system.xy_stage.set_xy(xy)
ce_system.xy_stage.set_rel_y(-0.01)

"""
635, 169
527, 191
436, 206
436, 115
436, 211
436, 314

"""

'\n635, 169\n527, 191\n436, 206\n436, 115\n436, 211\n\n\n'

In [44]:
scan_radius=0.5
ce_system.xy_stage.set_rel_xy(np.random.uniform(low=-scan_radius, high=scan_radius,size= 2))


In [88]:
ce_system.camera.continuous_snap()

True

In [9]:
import numpy as np
from scipy.ndimage import label, distance_transform_edt
from skimage import io, img_as_float, filters, morphology, transform, draw
from skimage.feature import peak_local_max
from skimage.measure import regionprops
from skimage.morphology import watershed
#from L4.image_util import *
import pandas as pd
import glob

class ImageDetect:
    
    """
    Detect whether there are cells located on the image,
    and locate the distance needed to move to center over a lysis spot.
    
    Image detect rescales images to 50%, be cautious of this as you compare raw or other transformed images.
    
    Example Usage:
    # files is a list of image files for testing purposes. 
    # Get Laser Position Information
    move_to = (200,400) # Pixel coordinates for the lysis position
    im_size = image_display.shape #should be the size of image used to determine pixel coordinates
    
    # Convert to general fractions of the image (for changing image sizes)
    move_to_frac = np.divide(move_to, im_size)
    lysis_radius_frac = 20/im_size[0]
    
    # This will need to be for the image at 0.5 scaled image
    mm_per_pix = [1, 1] * 1 # Change inside the brackets to flip an axis, change outside the axis to change the scalar

    # Keep searching images until you find one that will work.
    wait= True
    while wait:
        file = np.random.choice(files)
        print(file)
        img = io.imread(file)
        imd = ImageDetect(img)
        try:
            imd.get_regions()
            imd.select_cell()
            wait=False
        except AssertionError as e:
            print(e)

    imd.draw_cell()
    lysis_yx, lysis_rad = imd.draw_lysis(move_to_frac, lysis_radius_frac)
    imd.draw_vector(lysis_yx, lysis_rad, mm_per_pix)
    """
    
    def __init__(self, img):
        self.img = img.copy()
        self.img_display = None
        self.regions=None
        self.labels=None
        self.row = None
        self.create_blank_display()
        self.signal_image = None
        self.normalized_image = None
        self.filtered_image = None
        self.edge_sobel = None
        self.binary_otsu = None
        self.closed_image1 = None
        self.filled = None
        self.opened = None
        self.closed_image2 = None
        self.labels = None
        self.opened2 = None
        
    
    def get_regions(self):
        image = self.img
        # Make sure types are the same
        input_image = img_as_float(image)
        input_image = input_image 
        # Subtract background
        signal_image = input_image# - background

        #Resize for faster analysis
        signal_image = transform.rescale(signal_image, 0.5)

        # Normalize between 0 and 1
        self.normalized_image = normalized_image = (signal_image - signal_image.min()) / (signal_image.max() - signal_image.min())
        # Filter Image
        self.filtered_image = filtered_image = filters.median(normalized_image, behavior='ndimage')
        # Edge Detection
        self.edge_sobel = edge_sobel = filters.sobel(filtered_image)

        # Threshold
        thresh = filters.threshold_otsu(edge_sobel)
        self.binary_otsu = binary_otsu = edge_sobel > thresh

        # Binary Morphology Operations
        structure_element = morphology.disk(5)

        self.closed_image1 = closed_image = morphology.binary_closing(binary_otsu, structure_element)
        self.filled = filled = morphology.remove_small_holes(closed_image, area_threshold=200)
        self.opened = opened = morphology.remove_small_objects(filled, min_size=50)
        structure_element = morphology.disk(20)
        self.closed_image2 = closed_image = morphology.binary_closing(opened, structure_element)
        structure_element = morphology.disk(20)
        self.opened2 = opened_image = morphology.binary_opening(closed_image, structure_element)

        # Watershed
        distance = distance_transform_edt(opened_image)
        local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((50,50)), labels=opened_image)
        markers = label(local_maxi)[0]
        self.labels = labels = watershed(-distance, markers, mask=opened_image)

        #Regions
        assert np.max(labels)-np.min(labels)>=1, "No Cells Detected, Move and Try again"
            
        properties = ['centroid', 'area', 'eccentricity', 'convex_area', 'equivalent_diameter']
        regions = regionprops_table(labels,properties=properties)
        self.regions = regions = pd.DataFrame(regions)
        self.labels = labels
        return regions
    
    def select_cell(self):
        regions = self.regions

        # Filter the regions by area
        regions=regions[regions['area']>10000]
        assert regions.shape[0]>1, "No Cells Detected, Move and Try again"

        # Get a random row from the regions
        row = regions.sample(1).iloc[0]
        self.row = row
        return row
    
    def create_blank_display(self):
        # Normalize the image
        img=self.img.copy()
        img_display = transform.rescale(img, 0.5)
        im_st = img_display - np.quantile(img_display,0.1)
        #Convert to color
        rggb = gray2rgb(im_st*1/(np.quantile(im_st,0.95)))
        self.img_display=rggb
        return rggb
    
    def draw_cell(self):
        img_display = self.img_display[:,:,0]
        rggb = self.img_display
        row = self.row
        
        # Display our detected images:
        # Draw a circle around the selected cell
        rr, cc = draw.circle_perimeter(int(row['centroid-0']), int(row['centroid-1']),
                              int(row['equivalent_diameter']/2), shape=img_display.shape)

        # Get the circle perimeter of the cell and make it bigger
        perim = np.zeros(img_display.shape)
        perim[rr,cc]=1
        perim = morphology.dilation(perim, selem=morphology.disk(5))
        # Add it to the color image for the perimeter
        rggb[perim>0, :] = [1,0,0]
        return rggb

    def draw_lysis(self, move_to_frac, lysis_radius_frac):
        img_display = self.img_display[:,:,0]
        rggb = self.img_display
        # Read back the magic :D 
        lysis_yx = np.multiply(move_to_frac, img_display.shape).astype(int)
        lysis_radius = int(lysis_radius_frac*img_display.shape[0])
        # Get the Circle
        rr, cc = draw.circle(lysis_yx[0], lysis_yx[1], lysis_radius)
        rggb[rr,cc, :]=[0,1,0]
        return lysis_yx, lysis_radius
    
    def draw_vector(self, lysis_yx, lysis_radius, mm_per_pix):
        # Draw the Movement Vector
        img_display = self.img_display[:,:,0]
        rggb = self.img_display
        row = self.row
        # Get the difference between the coords
        cell_yx = [int(row['centroid-0']), int(row['centroid-1'])]
        cell_yx[1]+= int(row['equivalent_diameter']/4)
        diff_in_pixels = np.subtract(lysis_yx, cell_yx )
        # Draw a line
        rr,cc = draw.line(cell_yx[0], cell_yx[1], cell_yx[0]+diff_in_pixels[0], cell_yx[1]+diff_in_pixels[1])
        perim = np.zeros(img_display.shape)
        perim[rr,cc]=1
        perim = morphology.dilation(perim, selem=morphology.disk(5))
        rggb[perim>0, :] = [0,0,1]
        #Scalar to convert to mm
        diff_in_mm = diff_in_pixels*mm_per_pix
        print(f'From: {cell_yx} to {lysis_yx} in pixels')
        print(f'Pixel Distance = {diff_in_pixels} (y,x)')
        print(f'mm distance = {diff_in_mm} (y,x)')
        return diff_in_mm

In [13]:
reload(CellDetection)

scan_radius = 0.5
_focus_adjust = -.01
move_to = (800,250) # Pixel coordinates for the lysis position
# (1040, 1392)
im_size = (1040, 1392) # ce_system.camera.get_last_image().shape #should be the size of image used to determine pixel coordinates
# Convert to general fractions of the image (for changing image sizes)
move_to_frac = np.divide(move_to, im_size)
lysis_radius_frac = 20/im_size[0]
# This will need to be for the image at 0.5 scaled image
# For every 10 microns we move 99.5 pixels
# 
mm_per_pix = np.multiply([-1, -1], 0.01/99.5) # Change inside the brackets to flip an axis, change outside the axis to change the scalar
auto_lysis_dir = r"G:\My Drive\ForLuke\AutoLysisPt3"
note_file = auto_lysis_dir +r"\notes.csv"
import datetime
def save_a_note(prefix, msg, val, file):
    with open(file,'a') as fout:
        fout.write(f"{datetime.datetime.now()},{prefix},{msg},{val}\n")

no_cell = True
fig, ax = plt.subplots()
ce_system.camera.stop()

while no_cell:
    # Move Randomly
    #ce_system.xy_stage.set_rel_xy(np.random.uniform(low=-scan_radius, high=scan_radius,size= 2))
    time.sleep(.2)

    #Focus the Cells
    rsc = Focus.RefinedStepClimb(ce_system, Focus.brenner)
    bf_climb_focus = rsc.climb(initial_step = 0.05, final_step=0.002, refinements=3)
    ce_system.objective.set_z(bf_climb_focus + _focus_adjust)
    save_a_note("Auto",f"{counter}", f"{bf_climb_focus}", note_file)

    # Grab the Image
    img = img_as_float(ce_system.camera.snap())
    imd = ImageDetect(img)

    #See if there are any Cells, If not it will catch the assertion error and repeat
    try:
        imd.get_regions()
        imd.select_cell()
        no_cell=False
    except AssertionError as e:
        print(e)
    break

ce_system.camera.continuous_snap()  
        
# Once Cell has been found, create the outlines and move forward
imd.draw_cell()
lysis_yx, lysis_rad = imd.draw_lysis(move_to_frac, lysis_radius_frac)
imd.draw_vector(lysis_yx, lysis_rad, mm_per_pix)
# Display the image for the user
ax.imshow(imd.img_display)
plt.show()
save_a_note("CELLINFO", "SELECTED", f"{imd.row}".replace('\n', '~'), note_file)
# Wait for user to confirm
 
# Move the Stage to the relative position


C:\Users\NikonTE300CE\miniconda3\envs\CEpy37\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


HEYYOO


<ipython-input-9-68a903e2044d>:166: FutureWarning: circle is deprecated in favor of disk.circle will be removed in version 0.19
  rr, cc = draw.circle(lysis_yx[0], lysis_yx[1], lysis_radius)


From: [392, 163] to [400 124] in pixels
Pixel Distance = [  8 -39] (y,x)
mm distance = [-0.00080402  0.0039196 ] (y,x)


In [11]:
from pathlib import Path
import numpy as np
from scipy.ndimage import label, distance_transform_edt
from skimage import io, img_as_float, filters, morphology, draw, transform
from skimage.color import gray2rgb
from skimage.feature import peak_local_max
from skimage.measure import regionprops
from skimage.morphology import watershed



In [137]:
plt.imshow(markers)
plt.imshow(imd.markers)

TypeError: Invalid shape (696,) for image data

In [14]:
brae_idx = 0

## When it doesn't focus a cell correctly (no cell in view) Run the Below code before rerunning

In [64]:
ce_system.xy_stage.set_xy([-24.494, 39.283])
ce_system.objective.set_z(0.0785)

C:\Users\NikonTE300CE\miniconda3\envs\CEpy37\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
<ipython-input-9-68a903e2044d>:166: FutureWarning: circle is deprecated in favor of disk.circle will be removed in version 0.19
  rr, cc = draw.circle(lysis_yx[0], lysis_yx[1], lysis_radius)


From: [384, 276] to [400 124] in pixels
Pixel Distance = [  16 -152] (y,x)
mm distance = [-0.00160804  0.01527638] (y,x)
HEYYOO
HEYYOO
Cell Focus and Lysis taking Place...Wait
Fire
Injection Complete 5.04047417640686
Saving Data...
Data Saved... Find the next cell somewhere else on the slide


# If the Camera gives a "JAVA... Circular Buffer Error", Run the Below Code (after exiting the old camera view window)

In [57]:
# Microscope Display
cam = DisplayWindows.PLTMicroscopeDisplay(ce_system)
cam.scalar=1
ce_system.camera.stop()
cam.show()
cam.live_image()

HEYYOO


# Run this below code for testing

In [63]:
from L4 import Focus
import datetime
# Reload the L4 Files for any updates/changes
reload(Focus)
reload(image_util)
reload(CellDetection)

# File Saving Parameters
auto_lysis_dir = r"G:\My Drive\ForLuke\AutoLysisPt3"
note_file = auto_lysis_dir +r"\notes.csv"

# Injection Parameters
gravity_drop, voltage_level, injection_time  = [0, 5, 5]
delay=0.5
drift = 0

#Cell Searching Parameters
scan_radius=0.05 # mm 

# Laser Positioning Parameters
# Get Laser Position Information
move_to = (800,250) # Pixel coordinates for the lysis position
# (1040, 1392)
im_size = (1040, 1392) # ce_system.camera.get_last_image().shape #should be the size of image used to determine pixel coordinates
# Convert to general fractions of the image (for changing image sizes)
move_to_frac = np.divide(move_to, im_size)
lysis_radius_frac = 20/im_size[0]
mm_per_pix = np.multiply([-1, -1], 0.01/99.5) # Change inside the brackets to flip an axis, change outside the axis to change the scalar

#Focus Parameters
_focus_adjust = -0.01

# Set up the Objects Needed to Run
ims = image_util.ImageSaver(ce_system,folder_prefix='lysis', data_folder=auto_lysis_dir )

print("Testing Cell {counter}")
def save_slides(slide_show, *args):
    warnings.filterwarnings("ignore")
    slide_show.add_slide(*args)
    slide_show.save_presentation()
    warnings.filterwarnings("default")
    
def save_a_note(prefix, msg, val, file):
    with open(file,'a') as fout:
        fout.write(f"{datetime.datetime.now()},{prefix},{msg},{val}\n")

def finish_auto_lysis(ce_system):
    global counter
    ce_system.objective.set_rel_z(0.05)
    time.sleep(3)
    ce_system.inlet_z.set_rel_z(5)
    ce_system.objective.set_rel_z(-0.05)
    time.sleep(3)
    ims.remove_callback()    
    ims.save_image('IMG',folder_prefix="AutoLysis")
    ims.save_gif("AutoLysis", auto_lysis_dir)
    print("Data Saved... Find the next cell somewhere else on the slide")
    counter+=1
    
def limited_fire(ce_system, gravity_drop, voltage_level, injection_time):
    time.sleep(delay)
    print('Fire')
    ce_system.outlet_z.set_rel_z(-gravity_drop)
    ce_system.high_voltage.set_voltage(voltage_level)
    ce_system.outlet_pressure.release()
    ce_system.high_voltage.start()
    start_time = time.time()
    time.sleep(0.1)
    ce_system.lysis_laser.laser_standby()
    ce_system.lysis_laser.laser_fire()

    first=False
    while time.time()-start_time < injection_time:
        time.sleep(0.1)
        if time.time()-start_time >2 and first:
            ce_system.objective.set_rel_z(-_focus_adjust)
            time.sleep(0.2)
            ce_system.lysis_laser.laser_fire()
            first=False
        
    ce_system.high_voltage.stop()
    print(f"Injection Complete {time.time()-start_time}")
    ce_system.outlet_z.set_rel_z(gravity_drop)
    ce_system.outlet_pressure.seal()
    print("Saving Data...")
    finish_auto_lysis(ce_system)
    

fig, ax = plt.subplots()
    
resp=True
def run_me (user_input=True):
    def move_to_cell():
        no_cell = True
        save_a_note("CELLINFO", "RUNNING", f"START".replace(',', ' '), note_file)

        while no_cell:
            # Move Randomly
            ce_system.xy_stage.set_rel_xy(np.random.uniform(low=-scan_radius, high=scan_radius,size= 2))
            time.sleep(1.5)
            pre_xy = ce_system.xy_stage.read_xy()
            save_a_note("POSITIONINFO", "XY PRE", f"{pre_xy}".replace(',', ' '), note_file)
            pre_focus_start = ce_system.objective.read_z()
            save_a_note("POSITIONINFO", "OBJ PRE", f"{pre_focus_start}".replace(',', ' '), note_file)

            
            #Focus the Cells
            rsc = Focus.RefinedStepClimb(ce_system, Focus.brenner)
            bf_climb_focus = rsc.climb(initial_step = 0.075, final_step=0.002, refinements=4)
            ce_system.objective.set_z(bf_climb_focus)
            save_a_note("FOCUS",f"{counter}", f"{bf_climb_focus}", note_file)
            focus_start = ce_system.objective.read_z()
            save_a_note("POSITIONINFO", "OBJ POST", f"{focus_start}".replace(',', ' '), note_file)

            # Grab the Image
            img = img_as_float(ce_system.camera.snap())
            imd = ImageDetect(img)
            
            
            #See if there are any Cells, If not it will catch the assertion error and repeat
            try:
                imd.get_regions()
                imd.select_cell()
                no_cell=False
            except AssertionError as e:
                print(e)
                save_a_note("CELLINFO", "RUNNING", f"BROKEN".replace(',', ' '), note_file)
                ce_system.objective.set_z(pre_focus_start)
        ce_system.objective.set_rel_z(_focus_adjust)
        focus_start = ce_system.objective.read_z()

        # Once Cell has been found, create the outlines and move forward
        imd.draw_cell()
        lysis_yx, lysis_rad = imd.draw_lysis(move_to_frac, lysis_radius_frac)
        mm_move_y_x = imd.draw_vector(lysis_yx, lysis_rad, mm_per_pix)
        # Display the image for the user
        #ax.imshow(imd.img_display)
        xy = ce_system.xy_stage.read_xy()
        save_a_note("POSITIONINFO", "XY POST", f"{xy}".replace(',', ' '), note_file)
        save_a_note("CELLINFO", "SELECTED", f"{imd.row}".replace('\n', '~'), note_file)
        # Wait for user to confirm
  
        # Move the Stage to the relative position
        ce_system.xy_stage.set_rel_xy(mm_move_y_x[::-1] )

    resp=True
    if resp:
        # on reruns, you just need to run this command when you have the cells in focus
        resp = get_yes_no("Are there cells in the field of view? Do you want to autofocus?")
        drift = +0.005
        if resp:
            # Start Recording
            ce_system.camera.stop()
            ims.add_callback()
            # Find a Cell
            lysis_attempts = 0 
            move_to_cell()
            
            #Return so user can see updates
            ce_system.camera.continuous_snap()
            time.sleep(1)
            obj_height = ce_system.objective.read_z()
            cap_z = auto_utils.get_cap_height(obj_height, cap_difference)
            z = ce_system.inlet_z.read_z()

            rel_z = cap_z-z
            rel_z+=drift
            ce_system.inlet_z.set_rel_z(rel_z+0.015)
            save_a_note("Cap",f"{counter}", f"{rel_z} {_focus_adjust} {brae_idx}", note_file)
            #ce_system.inlet_z.wait_for_move()
            ce_system.objective.set_rel_z(-0.02)
            time.sleep(8)
            ce_system.camera.stop()
            rsc = Focus.RefinedStepClimb(ce_system, Focus.brenner)
            bf_climb_focus = rsc.climb(initial_step = 0.01, final_step=0.0002, refinements=3)
            ce_system.objective.set_z(bf_climb_focus)
            ce_system.inlet_z.set_rel_z(-0.015)
            time.sleep(2)

            ce_system.camera.continuous_snap()
            import threading
            #pulses+=1
            #lysis_attempts+=1
            threading.Thread(target=limited_fire, args=(ce_system, gravity_drop, voltage_level, injection_time)).start()
            print("Cell Focus and Lysis taking Place...Wait")

        else:
            print("Bring cells into focus.")
            assert 1==0, "Cells not in focus!!"
threading.Thread(target=run_me).start()
brae_idx +=1

Testing Cell {counter}
Are there cells in the field of view? Do you want to autofocus? y


C:\Users\NikonTE300CE\miniconda3\envs\CEpy37\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


No Cells Detected, Move and Try again


C:\Users\NikonTE300CE\miniconda3\envs\CEpy37\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


No Cells Detected, Move and Try again


...

No Cells Detected, Move and Try again


C:\Users\NikonTE300CE\miniconda3\envs\CEpy37\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


No Cells Detected, Move and Try again
OH NO ['R']


C:\Users\NikonTE300CE\miniconda3\envs\CEpy37\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


No Cells Detected, Move and Try again
OH NO ['R']


### For Auto Focus test part two, you will need to find the cells on the slide.

Use a capillary you don't care about just in case something goes wrong. Does not need a detection window if thats easier (Does need a clean inlet)

We don't need to adjust the focus of the objective finely at all. All we care about is replicating the conditions. 

We will need 30 cells to get a good idea of its performance. It doesn't need to run the separation. 

Once you have calibrated the stages and capillary, you should be good to run the scripts. 

### Test the Auto Focus here!
Find cells of interest and focus them before running the next command. Auto focus, capillary lowering, and lysis will follow. 

Repeat for 30 cells, one after the other. No separation needed yet. 

## This is what we have previously done. You may need to verify this on your own. 

In [278]:
reload(image_util)
auto_lysis_dir = r"G:\My Drive\ForLuke\AutoLysisPt2"
note_file = auto_lysis_dir +r"\notes.csv"
gravity_drop, voltage_level, injection_time  = [0, 5, 5]
delay=0.5
drift = 0
ims = image_util.ImageSaver(ce_system,folder_prefix='lysis', data_folder=auto_lysis_dir )
if np.random.random() >0.:
    _focus_adjust = -0.01
else:
    _focus_adjust = -0.01

print("Testing Cell {counter}")
from L4 import Focus
def save_slides(slide_show, *args):
    warnings.filterwarnings("ignore")
    slide_show.add_slide(*args)
    slide_show.save_presentation()
    warnings.filterwarnings("default")
    
def save_a_note(prefix, msg, val, file):
    with open(file,'a') as fout:
        fout.write(f"{prefix},{msg},{val}\n")


def finish_auto_lysis(ce_system):
    global counter
    ce_system.objective.set_rel_z(0.05)
    time.sleep(3)
    ce_system.inlet_z.set_rel_z(5)
    ce_system.objective.set_rel_z(-0.05)
    time.sleep(3)
    ims.remove_callback()    
    ims.save_image('IMG',folder_prefix="AutoLysis")
    ims.save_gif("AutoLysis", auto_lysis_dir)
    print("Data Saved... Find the next cell somewhere else on the slide")
    counter+=1
    
def limited_fire(ce_system, gravity_drop, voltage_level, injection_time):
    time.sleep(delay)
    print('Fire')
    ce_system.outlet_z.set_rel_z(-gravity_drop)
    ce_system.high_voltage.set_voltage(voltage_level)
    ce_system.outlet_pressure.release()
    ce_system.high_voltage.start()
    start_time = time.time()
    time.sleep(0.1)
    ce_system.lysis_laser.laser_standby()
    ce_system.lysis_laser.laser_fire()

    first=False
    while time.time()-start_time < injection_time:
        time.sleep(0.1)
        if time.time()-start_time >2 and first:
            ce_system.objective.set_rel_z(-_focus_adjust)
            time.sleep(0.2)
            ce_system.lysis_laser.laser_fire()
            first=False
    ce_system.high_voltage.stop()
    print(f"Injection Complete {time.time()-start_time}")
    ce_system.outlet_z.set_rel_z(gravity_drop)
    ce_system.outlet_pressure.seal()
    print("Saving Data...")
    finish_auto_lysis(ce_system)

    
resp=True
def run_me ():
    resp=True
    if resp:
        # on reruns, you just need to run this command when you have the cells in focus
        resp = get_yes_no("Are there cells in the field of view? Do you want to autofocus?")
        drift = +0.005
        if resp:
            # Start Recording
            ce_system.camera.stop()
            ims.add_callback()
            lysis_attempts = 0 

            rsc = Focus.RefinedStepClimb(ce_system, Focus.brenner)
            bf_climb_focus = rsc.climb(initial_step = 0.01, final_step=0.0002, refinements=3)
            ce_system.objective.set_z(bf_climb_focus + _focus_adjust)
            save_a_note("Auto",f"{counter}", f"{bf_climb_focus}", note_file)
            ce_system.camera.continuous_snap()
            time.sleep(1)
            obj_height = ce_system.objective.read_z()
            print(obj_height)
            cap_z = auto_utils.get_cap_height(obj_height, cap_difference)
            z = ce_system.inlet_z.read_z()

            rel_z = cap_z-z
            rel_z+=drift
            ce_system.inlet_z.set_rel_z(rel_z+0.015)
            save_a_note("Cap",f"{counter}", f"{rel_z} {_focus_adjust} {brae_idx}", note_file)
            #ce_system.inlet_z.wait_for_move()
            ce_system.objective.set_rel_z(-0.02)
            time.sleep(8)
            ce_system.camera.stop()
            rsc = Focus.RefinedStepClimb(ce_system, Focus.brenner)
            bf_climb_focus = rsc.climb(initial_step = 0.01, final_step=0.0002, refinements=3)
            ce_system.objective.set_z(bf_climb_focus)
            ce_system.inlet_z.set_rel_z(-0.015)
            time.sleep(2)

            ce_system.camera.continuous_snap()
            import threading
            #pulses+=1
            #lysis_attempts+=1
            threading.Thread(target=limited_fire, args=(ce_system, gravity_drop, voltage_level, injection_time)).start()
            print("Cell Focus and Lysis taking Place...Wait")

        else:
            print("Bring cells into focus.")
            assert 1==0, "Cells not in focus!!"
threading.Thread(target=run_me).start()
brae_idx +=1

Testing Cell {counter}
Are there cells in the field of view? Do you want to autofocus? y
HEYYOO
0.0271
Data Saved... Find the next cell somewhere else on the slide
HEYYOO
Cell Focus and Lysis taking Place...Wait
Fire
Injection Complete 5.0412750244140625
Saving Data...
Data Saved... Find the next cell somewhere else on the slide


In [159]:
ce_system.inlet_z.set_rel_z(1)

In [343]:
ce_system.xy_stage.set_rel_xy(np.random.random(2))

In [344]:
lysis_pos = [800,300] # Row Column

In [346]:
xy = ce_system.xy_stage.read_xy()

In [386]:
ce_system.xy_stage.set_xy(xy)

In [407]:
# get background
from skimage import io, img_as_float, filters, morphology
from tqdm.notebook import tqdm
#ce_system.objective.set_rel_z(-1)
ce_system.camera.stop()
imgs = []
st = time.time()
for i in tqdm(range(50)):
    ce_system.xy_stage.set_rel_xy(np.random.uniform(low=-.5, high=0.5,size= 2))
    time.sleep(.2)
    imgs.append( img_as_float(ce_system.camera.snap()))
ce_system.camera.continuous_snap()
imgs = np.asarray(imgs)
median_img = np.median(imgs[:50,:,:],axis=0)
#ce_system.objective.set_rel_z(1)


  0%|          | 0/50 [00:00<?, ?it/s]

HEYYOO


In [468]:
from L4 import CellDetection as cd
reload(cd)
img = ce_system.camera.get_last_image()
cd.get_blobs(img, 'scikit_threshold', mean_img, 10, 50, (10,10))

[] [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


C:\Users\NikonTE300CE\miniconda3\envs\CEpy37\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


[]

In [382]:
import numpy as np
from scipy.ndimage import label, distance_transform_edt
from skimage import io, img_as_float, filters, morphology
from skimage.feature import peak_local_max
from skimage.measure import regionprops
from skimage.morphology import watershed
#from L4.image_util import *
import pandas as pd

In [472]:
plt.figure()
plt.imshow(labels)

In [469]:
image = img
imgs = np.asarray(imgs)
mean_img = np.mean(imgs[:50,:,:],axis=0)
background = img_as_float(mean_img)
background = background 
# Make sure types are the same
input_image = img_as_float(image)
input_image = input_image 
# Subtract background
signal_image = input_image - background

# Normalize between 0 and 1
normalized_image = (signal_image - signal_image.min()) / (signal_image.max() - signal_image.min())
# Filter Image
filtered_image = filters.median(normalized_image, behavior='ndimage')
# Edge Detection
edge_sobel = filters.sobel(filtered_image)

# Threshold
thresh = filters.threshold_otsu(edge_sobel)
binary_otsu = edge_sobel > thresh

# Binary Morphology Operations
structure_element = morphology.disk(10)
closed_image = morphology.binary_closing(binary_otsu, structure_element)
structure_element = morphology.disk(50)
opened_image = morphology.binary_opening(closed_image, structure_element)
# Watershed
distance = distance_transform_edt(opened_image)
local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((50,50)), labels=opened_image)
markers = label(local_maxi)[0]
labels = watershed(-distance, markers, mask=opened_image)

C:\Users\NikonTE300CE\miniconda3\envs\CEpy37\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


In [118]:
_focus_adjust

0.015

1. Go to random spot
2. Focus
3. Lower cap
4. Raise Obj (at steps) up to 0.1


In [415]:
ce_system.camera.stop()
rsc = Focus.RefinedStepClimb(ce_system, Focus.brenner)
bf_climb_focus = rsc.climb(initial_step = 0.03, final_step=0.0002, refinements=3)
ce_system.objective.set_z(bf_climb_focus)
ce_system.camera.continuous_snap()

HEYYOO


True

In [284]:
ce_system.objective.set_rel_z(-0.02)

In [281]:

obj_height = ce_system.objective.read_z()
print(obj_height)
cap_z = auto_utils.get_cap_height(obj_height, cap_difference)
z = ce_system.inlet_z.read_z()
rel_z = cap_z-z
rel_z+=drift
ce_system.inlet_z.set_rel_z(rel_z)

0.0116


In [341]:
# Lower the capillary (ADJUSTMENT)
# If desired, add this to your drift to keep the capillary in the correct distance above the cell focus. 
ce_system.inlet_z.set_rel_z(3)

### Lyse the Cell and load the lysate into the capillary 

In [295]:
# Start the Single Cell Injection

# General Injection Parameters
injection_time = 3 # in seconds
gravity_drop = 0 # in millimeters
voltage_level = 5 # in kilovolts
delay = 1 # inseconds

_focus_adjust = 0.033257
from L4 import Focus
rsc = Focus.RefinedStepClimb(ce_system, Focus.brenner)
bf_climb_focus = rsc.climb(initial_step = 0.005, final_step=0.0002, refinements=3)
ce_system.objective.set_rel_z(_focus_adjust)


import threading

def finish_auto_lysis(ce_system):
    
    ce_system.objective.set_rel_z(0.05)
    time.sleep(3)
    ce_system.inlet_z.set_rel_z(5)
    ce_system.objective.set_rel_z(-0.05)
    time.sleep(3)
    ims.remove_callback()    
    last_lysis = ims.save_image('IMG',folder_prefix="AutoLysis")

#pulses+=1
def limited_fire(ce_system, gravity_drop, voltage_level, injection_time):
        
    time.sleep(delay)
    print('Fire')
    ce_system.outlet_z.set_rel_z(-gravity_drop)
    ce_system.high_voltage.set_voltage(voltage_level)
    ce_system.outlet_pressure.release()
    ce_system.high_voltage.start()
    start_time = time.time()
    time.sleep(0.1)
    ce_system.lysis_laser.laser_standby()
    ce_system.lysis_laser.laser_fire()
    
    first=False
    while time.time()-start_time < injection_time:
        time.sleep(0.1)
        if time.time()-start_time >10 and first:
            ce_system.lysis_laser.laser_fire()
            first=False
    ce_system.high_voltage.stop()
    print(f"Finished {time.time()-start_time}")
    ce_system.outlet_z.set_rel_z(gravity_drop)
    ce_system.outlet_pressure.seal()
    
    finish_auto_lysis()
    

#lysis_attempts+=1
threading.Thread(target=limited_fire, args=(ce_system, gravity_drop, voltage_level, injection_time)).start()

#threading.Thread(target=fire_me, args=arg).start()

Fire
Finished 3.028820037841797


In [189]:
# Run this to push out ECB buffer if you want to redo injections
ce_system.outlet_pressure.rinse_pressure()
time.sleep(10)
ce_system.outlet_pressure.release()

In [190]:
#Move the capillary up to see better
ce_system.inlet_z.set_rel_z(0.02)

### Continue to the separation (or whatever the next step in the method is) 

In [296]:
# Continue the Method
def go_ahead():
    global last_lysis 
    ce_system.objective.set_rel_z(0.05)
    time.sleep(2)
    ce_system.inlet_z.set_rel_z(3)
    ce_system.objective.set_rel_z(-0.05)
    time.sleep(2)
    ce_system.objective.set_rel_z(-5)
    if not auto_program.continue_event.is_set():
        
        auto_program.continue_event.set()
    else:
        auto_program.start_run()
        
    if record_lysis:
        ims.remove_callback()
        
        last_lysis = ims.save_gif('Lysis', None, scale=0.25)
    
threading.Thread(target=go_ahead).start()

After special True
Time to wait is  3
After special True
Time to wait is  240


### Add any notes about the cell or injection that you wish to record here. 

If you are collecting NA it is a good idea to record which pcr tube you are collecting this into. 
ie (Strip A, Well 1) 

In [297]:
cell_notes = 'blank injection'

XY Errr ['']
After special True
Time to wait is  90
XY Errr ['']
After special True
Time to wait is  45


# misc functions

In [194]:
# Record the previous path as the last path, this should be run when restarting the auto_program object
auto_program.last_path = file_path
file_path

'.\\Data\\Pinocytosis C24C ECB_00024.csv'

In [298]:
# Run this when you have finished the last repetition
file_path = r"C:\Users\NikonTE300CE\Documents\Data\24C SCA 022821.csv" #str(auto_program.last_path)
cell_notes = f'EK Injection: {voltage_level} kV, {injection_time} s, Pulses required: {pulses} \n' + cell_notes
threading.Thread(target=save_slides, args=(slide_show, file_path,last_lysis, None, cell_notes)).start()
auto_program.last_path=None
pulses=0

Exception in thread Thread-168622:
Traceback (most recent call last):
  File "C:\Users\NikonTE300CE\miniconda3\envs\CEpy37\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\NikonTE300CE\miniconda3\envs\CEpy37\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-289-0af344fc7e7d>", line 7, in save_slides
  File "C:\Users\NikonTE300CE\Desktop\Barracuda_Updated\AutomatedCE\L4\FileIO.py", line 168, in add_slide
    pks_rfu, pks_time = generate_table(egram_filename, slide, x=pad, y=graph_h + pad, cx=graph_w,
  File "C:\Users\NikonTE300CE\Desktop\Barracuda_Updated\AutomatedCE\L4\FileIO.py", line 228, in generate_table
    data = pd.read_csv(data_file)
  File "C:\Users\NikonTE300CE\miniconda3\envs\CEpy37\lib\site-packages\pandas\io\parsers.py", line 605, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "C:\Users\NikonTE300CE\miniconda3\envs\CEpy37\lib\site-packages\pandas\io\parsers.py", line 457, 

In [299]:
# Save the presentation 
slide_show.save_presentation()

In [302]:
ce_system.outlet_pressure.rinse_pressure()

In [303]:
ce_system.outlet_pressure.release()

In [307]:
ce_system.objective.set_rel_z(-5)

After special True
Time to wait is  120
After special True
Time to wait is  45


After special True
Time to wait is  45


C:\Users\NikonTE300CE\Desktop\Barracuda\AutomatedCE\L4\DisplayWindows.py:277: UserWarning: Attempting to set identical left == right == 0.0 results in singular transformations; automatically expanding.
  ax1.set_xlim(min(data['time_data']), max(data['time_data']))


After special True
Time to wait is  45


In [456]:
ce_system.outlet_pressure.seal()

In [53]:
ce_system.inlet_z.read_z()

0.8488835

In [37]:
ce_system.inlet_z.set_z(24)